# Using Azure OpenAI GPT models

This sample demonstrates how to get started with Azure OpenAI Chat Completions.

In [26]:
%pip install openai --quiet
%pip install python-dotenv --quiet
%pip install numpy --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Load the environment variables from the `.env` file to connect to the LLM model.

In [16]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

## Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [23]:
import os
from openai import AzureOpenAI

endpoint = "https://ai-services-31293-400003.openai.azure.com/"
subscription_key = "EfbmGw4MsbvXhItjP6OdrJR6nbjoQj9dWU0FSOf1p26HBNKdXoeMJQQJ99BCACYeBjFXJ3w3AAAAACOGv7IZ"
deployment = "gpt-4o"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

# Prepare the chat prompt
messages = [
    {
        "role": "system", "content": "You are an AI assistant that helps people find information.",
    },
    {
        "role": "user", "content": "What is the capital of France?"
    },
]

response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    logprobs=True,
    top_logprobs=3
)

print(response.choices[0].message.content)

The capital of France is **Paris**.


Lets view the full JSON response.

In [24]:
# Print the full response object
print(response.to_json())

{
  "id": "chatcmpl-BD8ggz5NHJ8lfgjYpyIfv3a2BNhlE",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": {
        "content": [
          {
            "token": "The",
            "bytes": [
              84,
              104,
              101
            ],
            "logprob": 0.0,
            "top_logprobs": [
              {
                "token": "The",
                "bytes": [
                  84,
                  104,
                  101
                ],
                "logprob": 0.0
              },
              {
                "token": "Paris",
                "bytes": [
                  80,
                  97,
                  114,
                  105,
                  115
                ],
                "logprob": -17.0
              },
              {
                "token": " The",
                "bytes": [
                  32,
                  84,
                  104,
                  101
              

Let's view the probabilities of each word.

In [27]:
import numpy as np
from IPython.display import display, HTML

html_content = ""

for content in response.choices[0].logprobs.content:

    top_logprobs = content.top_logprobs

    for i, logprob in enumerate(top_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )

    html_content += f"<br>"
    
display(HTML(html_content))